<a href="https://colab.research.google.com/github/nikhilv7/MIMO-gao/blob/master/mimogao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow
from tensorflow import keras
import numpy as np
import random
import math

In [37]:
x = list(range(0, 20))
pi = np.pi
n = (np.random.normal(0, 1, [64, 1]) + 1j * np.random.normal(0, 1, [64, 1])) / np.sqrt(2)

phi = np.multiply((2 * pi / 20), random.choices(x, k=8))
phi.reshape([1, 8])
aphil=np.zeros([64,8],dtype=complex)
for i in range(0, 64):
    aphil[i] = np.exp(-1j * 2 * pi * np.sin(phi) * i)
aphil=aphil/np.sqrt(8)
h=np.sum(n*aphil,axis=1)
h_vector=np.concatenate((np.real(h),np.imag(h)),axis=0)
print(h_vector,h_vector.shape)

[ 5.25429726e-01  3.04609271e-01 -6.96119176e-01  7.45818211e-01
  1.13189969e-01  5.23103710e-02 -6.16684299e-02  5.78704047e-01
  1.97054598e+00 -2.45821113e-02  2.10847645e-01  1.90461777e+00
 -8.12587752e-01  1.63834204e-01 -5.21298651e-01  7.44642000e-01
 -3.96810763e-01 -1.51520619e+00 -5.74823968e-01  3.94589277e-01
 -6.47151184e-01 -4.27662898e-01  7.97328414e-01  1.02521622e-01
 -1.98869693e-01 -6.74994015e-01 -3.88462543e-01 -1.85235078e-01
  6.88337834e-01  1.01381144e+00  2.63404079e-02  2.01537836e-01
  6.83918646e-01  1.12122315e+00  9.91015925e-02 -2.35939896e-01
 -2.44803697e+00  3.33088368e-01  1.16931594e+00 -1.52027867e+00
 -8.03699366e-02  8.15669744e-01  1.27134947e+00  9.18551970e-02
  4.52013378e-01 -1.94922207e-02  1.46887759e+00 -6.84005919e-01
 -4.56402896e-01  1.31063735e+00  1.62609351e+00 -8.62135233e-01
 -2.34062675e-01  1.16369270e+00  6.01795474e-01  1.37825605e+00
  1.12572451e+00 -4.77938117e-01 -4.92052875e+00  2.04311559e-01
 -1.44081390e+00  1.78498

ADC model

>3 bit ADC



In [33]:
import os
n=0.5
alpha=0.96546
y= h+n
y1=y[:32]
y2=alpha*y[32:]

r=np.concatenate((y1,y2),axis=0)

r_real=np.real(r)
r_imag=np.imag(r)
r_vector=np.concatenate((r_real,r_imag),axis=0)
r_vector.reshape([128,1])
print(r)
print(r_vector)
print(r_vector.shape)

[ 0.04813384+0.32305368j  0.16647563+1.33653772j  0.89784345+0.15463082j
  0.1935609 -0.51599872j  1.50057199-0.48383384j  1.18798373+0.07990976j
  0.6547085 -0.71369098j  0.58715217+0.20185019j -0.83090988+0.41112478j
  0.32041204-0.65017509j  0.04388447-0.66385987j  0.4858046 -0.07838297j
  0.37558881+0.19473531j  0.25886987+0.15692288j  0.48047069+0.03664731j
 -0.11795336+0.73483025j -0.6650691 -0.71388407j  1.59604413+1.38037444j
  0.4580624 -0.02865153j -0.33923565+0.32919032j -0.55631124-0.38934576j
  1.29358649+0.11828459j  1.21273183+0.58261723j  0.34587017-0.51615094j
  0.37972184+0.10753743j  1.29153782+0.42271964j -1.19234009+0.38275134j
 -0.10200951-0.05251141j  1.00199884-1.13646911j  0.58918666+0.1032791j
 -0.08252295+0.18227583j  0.33696127+0.49730896j  1.14036318-0.58250396j
  0.52129852-0.00383207j  0.74715178-0.18110624j  0.4902689 -0.10800051j
 -0.95568085-0.89196797j  0.90624851+1.13000841j  0.4552435 +0.61572647j
  0.64535124+1.40069971j  0.32230262-0.00655807j  2.

Model for DI-DNN

In [39]:
model = keras.models.Sequential([
    keras.layers.Dense(160, activation='relu', input_shape=[128, 1]),
    keras.layers.Dense(200, activation='relu'),
    keras.layers.Dense(160, activation='relu'),
    keras.layers.Dense(128, activation='tanh'),
])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss='mse',metrics='accuracy')
model.summary()
model.fit(r_vector,h_vector,epochs=100,verbose=1)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 128, 160)          320       
_________________________________________________________________
dense_15 (Dense)             (None, 128, 200)          32200     
_________________________________________________________________
dense_16 (Dense)             (None, 128, 160)          32160     
_________________________________________________________________
dense_17 (Dense)             (None, 128, 128)          20608     
Total params: 85,288
Trainable params: 85,288
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
4/4 [==============================] - 0s 4ms/step - loss: 0.9017 - accuracy: 0.0000e+00
Epoch 2/100
4/4 [==============================] - 0s 4ms/step - loss: 0.8998 - accuracy: 0.0000e+00
Epoch 3/100
4/4 [=============================